In [ ]:
import numpy as np
import cv2
import math

In [ ]:
img = cv2.imread("Picture4-small_parial.bmp",0)

In [ ]:
def ClassifyImage(img,ClassA,ClassB,ClassC,blockSize,jumpSize):
    m,n = img.shape
    newImgA = np.copy(img)
    newImgB = np.copy(img)
    newImgC = np.copy(img)
    scoreA=0
    scoreB=0
    scoreC=0
    for i in range(0,m-int(blockSize/2),jumpSize):
        for j in range(0,n-int(blockSize/2),jumpSize):
            block = createBlocks(img,i,j,blockSize)
            if(compare(block,ClassA) > 80):
                scoreA+=1
                newImgA = createBoundingBox(newImgA,i,j,blockSize)
            if(compare(block,ClassB) > 75):
                scoreB+=1
                newImgB = createBoundingBox(newImgB,i,j,blockSize)
            if(compare(block,ClassC) > 80):
                scoreC+=1
                newImgC = createBoundingBox(newImgC,i,j,blockSize)
    cv2.imwrite("newImgA.jpg",newImgA)
    cv2.imwrite("newImgB.jpg",newImgB)
    cv2.imwrite("newImgC.jpg",newImgC)           
    print(scoreA,scoreB,scoreC)  

In [ ]:
def createBoundingBox(img,x,y,blockSize):
    if(x + blockSize > len(img)):
        blockSize = len(img) - x -1
    if(y + blockSize > len(img[0])):
        blockSize = len(img[0]) - y -1
    for i in range(x,x+blockSize):
        img[i][y] = 0
    for i in range(y,y+blockSize):
        img[x][i] = 0
    for i in range(x,x+blockSize):
        img[i][y+blockSize] = 0
    for i in range(y,y+blockSize):
        img[x+blockSize][i] = 0
    return img

In [ ]:
def createBlocks(img,x,y,blockSize):
    blocks=[]
    block=[]
    for j in range(0,blockSize):
        xblock=[]
        for i in range(0,blockSize):
            if(((x+j) < len(img)) and ((y+i) < len(img[0]))):
                xblock.append(img[x+j][y+i])
        block.append(xblock)
    blocks.append(block)
    return np.array(block)

In [ ]:
def createDCTMatrix(size):
    matrix = np.zeros((size,size))
    for i in range(0,len(matrix)):
        for j in range(0,len(matrix[0])):
            if(i == 0):
                matrix[i][j] = (1/size) ** 0.5
            else:
                term = ((2*j)+1)*math.pi*i
                term = term/(2*size)
                matrix[i][j] = (((2/size) **0.5) * math.cos(term))
    return matrix

In [ ]:
def compare(block1,block2):
    #histogram Analysis
    score = 0
    error = 0
    hist1 = make_dict_block(block1)
    hist2 = make_dict_block(block2)
    for i in hist1.keys():
        if(i in hist2):
            if((hist1[i]+15)>hist2[i]>(hist1[i]-15)):
                score+=1
            else:
                error -=1
        else:
            error -=1
    percentage1 = ((score + error) / (score - error)) *100
    
    #Sum of squared error
    b1 = sumOfBlocks(block1)
    b2 = sumOfBlocks(block2)
    percentage2 = (max(b1,b2)/ min(b1,b2)) *100

    #-----------------
    percentage = (0.7 * percentage1) + (0.3 * percentage2)
    return percentage


In [ ]:
def getMax(matrix, noOfPixel):
    m = []
    newMatrix = np.zeros((len(matrix),len(matrix[0])))
    for i in range(0,len(matrix)):
        for j in range(0,len(matrix[0])):
            newMatrix[i][j] = abs(matrix[i][j])

    while(len(m) != noOfPixel):
        maximum=0
        for i in range(0,len(matrix)):
            for j in range(0,len(matrix[0])):
                if(matrix[i][j] not in m):
                    if(newMatrix[i][j]>maximum):
                        maximum = matrix[i][j]
        m.append(maximum)
    return m 

In [ ]:
def sumOfBlocks(block):
    su = 0
    for i in block:
        for j in i:
            su +=j
    return su

In [ ]:
def make_dict_block(arr):
    d = dict()
    for i in arr:
        for j in i:
            if(j not in d.keys()):
                d[j] = 1
            else:
                d[j] +=1
    return d

In [ ]:
ClassA = (np.array(createBlocks(img,285,230,80)))
ClassB = (np.array(createBlocks(img,270,504,80)))
ClassC = (np.array(createBlocks(img,200,400,80)))
ClassifyImage(np.copy(img),ClassA,ClassB,ClassC,80,40)

18 13 2
